In [ ]:
import holoviews as hv; hv.extension('plotly','bokeh', logo=False)
import panel as pn;     pn.extension()

import numpy as np
from scipy.integrate import odeint

In [ ]:
nu = 0.001   # Diffusion constant
L  = 20      # Length of domain
N  = 1000    # Number of discretization points
dx = L/N
x  = np.arange(-L/2,L/2,dx) # Define x domain

# Define discrete wavenumbers
kappa = 2*np.pi*np.fft.fftfreq(N, d=dx)

# Initial condition
u0 = 1/np.cosh(x)

# Simulate PDE in spatial domain
dt = 0.025
t = np.arange(0,100*dt,dt)

def rhsBurgers(u,t,kappa,nu):
    uhat    = np.fft.fft(u)
    d_uhat  = (1j)*kappa*uhat
    dd_uhat = -np.power(kappa,2)*uhat
    d_u     = np.fft.ifft(d_uhat)
    dd_u    = np.fft.ifft(dd_uhat)
    du_dt   = -u * d_u + nu*dd_u
    return du_dt.real

u = odeint(rhsBurgers,u0,t,args=(kappa,nu))

In [ ]:
u_plot = u[0:-1:10,:]

hv.output( backend='plotly', size=200)
hv.Overlay([hv.Path3D((x,j*np.ones(u_plot.shape[1]),u_plot[j,:]), kdims=["x", "Time step"], vdims="Amplitude") for j in range(u_plot.shape[0])])

In [ ]:
hv.output(size=100)
hv.Raster(np.flipud(u))

In [ ]:
hv.output( backend='bokeh', holomap="scrubber")
hv.HoloMap({i: hv.Curve((x,u_plot[i,:]), "x" "Amplitude") for i in range(u_plot.shape[0])}, kdims='i')]

In [ ]:
player = pn.widgets.DiscretePlayer(name='Discrete Player', options=list(range(u_plot.shape[0])),value=0)
@pn.depends(player)
def show(value):
    return hv.Curve((x,u_plot[value,:]), "x" "Amplitude"),
pn.Column( show, player )